In [1]:
# Import the required modules
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

In [2]:
# Read in the transaction_fraud_data.csv file into a PandasDataFrame.
ten_year_data = pd.read_csv(Path("../Resources/treasury_yield_10yr.csv"), index_col='Date', infer_datetime_format=True, parse_dates=True)

#ten_year_data = ten_year_data.set_index('Date')

# Review the DataFrame
ten_year_data.head(10)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-10-31,2.370,2.379,2.369,2.376,2.376,0.0
2017-11-01,2.398,2.401,2.349,2.376,2.376,0.0
2017-11-02,2.363,2.370,2.336,2.347,2.347,0.0
2017-11-03,2.351,2.361,2.324,2.343,2.343,0.0
2017-11-05,NaN,NaN,NaN,NaN,NaN,NaN
2017-11-06,2.325,2.333,2.315,2.320,2.320,0.0
2017-11-07,2.327,2.329,2.304,2.307,2.307,0.0
2017-11-08,2.315,2.327,2.306,2.325,2.325,0.0
2017-11-09,2.333,2.345,2.315,2.331,2.331,0.0


In [3]:
ten_year_data = ten_year_data.drop(columns=["Open", "High", "Low", "Close", "Volume"])

ten_year_data.head()

,Adj Close
Date,
2017-10-31,2.376
2017-11-01,2.376
2017-11-02,2.347
2017-11-03,2.343
2017-11-05,NaN


In [4]:
ten_year_data = ten_year_data.dropna()

ten_year_data.head()

,Adj Close
Date,
2017-10-31,2.376
2017-11-01,2.376
2017-11-02,2.347
2017-11-03,2.343
2017-11-06,2.320


In [5]:
lags = 5

for i in range(0, lags):
	ten_year_data['Lag_' + str(i+1)] = ten_year_data['Adj Close'].shift(i+1)

ten_year_data['Returns'] = ten_year_data['Adj Close'].pct_change()

ten_year_data.head()

,Adj Close,Lag_1,Lag_2,Lag_3,Lag_4,Lag_5,Returns
Date,,,,,,,
2017-10-31,2.376,NaN,NaN,NaN,NaN,NaN,NaN
2017-11-01,2.376,2.376,NaN,NaN,NaN,NaN,0.000000
2017-11-02,2.347,2.376,2.376,NaN,NaN,NaN,-0.012205
2017-11-03,2.343,2.347,2.376,2.376,NaN,NaN,-0.001704
2017-11-06,2.320,2.343,2.347,2.376,2.376,NaN,-0.009816


In [6]:
# create the lagged percentage returns coluns
for i in range(0, lags):
	ten_year_data['Lag_' + str(i+1)] = ten_year_data["Lag_" + str(i+1)].pct_change()
	ten_year_data.fillna(0, inplace=True)
    
ten_year_data.head(10)

,Adj Close,Lag_1,Lag_2,Lag_3,Lag_4,Lag_5,Returns
Date,,,,,,,
2017-10-31,2.376,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-11-01,2.376,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-11-02,2.347,0.000000,inf,0.000000,0.000000,0.000000,-0.012205
2017-11-03,2.343,-0.012205,0.000000,inf,0.000000,0.000000,-0.001704
2017-11-06,2.320,-0.001704,-0.012205,0.000000,inf,0.000000,-0.009816
2017-11-07,2.307,-0.009816,-0.001704,-0.012205,0.000000,inf,-0.005603
2017-11-08,2.325,-0.005603,-0.009816,-0.001704,-0.012205,0.000000,0.007802
2017-11-09,2.331,0.007802,-0.005603,-0.009816,-0.001704,-0.012205,0.002581
2017-11-10,2.400,0.002581,0.007802,-0.005603,-0.009816,-0.001704,0.029601


In [7]:
# convert returns to the sign of direction
ten_year_data['Direction'] = np.sign(ten_year_data['Returns'])

ten_year_data = ten_year_data.dropna()
ten_year_data = ten_year_data.drop(ten_year_data.index[0:5])

##remove rows with any values that are not infinite
ten_year_data = ten_year_data[np.isfinite(ten_year_data).all(1)]

ten_year_data

,Adj Close,Lag_1,Lag_2,Lag_3,Lag_4,Lag_5,Returns,Direction
Date,,,,,,,,
2017-11-08,2.325,-0.005603,-0.009816,-0.001704,-0.012205,0.000000,0.007802,1.0
2017-11-09,2.331,0.007802,-0.005603,-0.009816,-0.001704,-0.012205,0.002581,1.0
2017-11-10,2.400,0.002581,0.007802,-0.005603,-0.009816,-0.001704,0.029601,1.0
2017-11-13,2.400,0.029601,0.002581,0.007802,-0.005603,-0.009816,0.000000,0.0
2017-11-14,2.381,0.000000,0.029601,0.002581,0.007802,-0.005603,-0.007917,-1.0
...,...,...,...,...,...,...,...,...
2022-10-24,4.234,-0.003076,0.023988,0.032266,-0.004234,0.001247,0.004985,1.0
2022-10-25,4.108,0.004985,-0.003076,0.023988,0.032266,-0.004234,-0.029759,-1.0
2022-10-26,4.015,-0.029759,0.004985,-0.003076,0.023988,0.032266,-0.022639,-1.0


In [8]:
X = ten_year_data[["Lag_1", "Lag_2"]]
y = ten_year_data["Direction"]

## Split into training and testing windows
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

start_test = pd.Timestamp("2017-11-14")

X_train = X[X.index < start_test]
X_test = X[X.index >= start_test]
y_train = y[y.index < start_test]
y_test = y[y.index >= start_test]

In [9]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [10]:
lr_accuracy = accuracy_score(y_test, y_pred)
lr_accuracy

0.500801282051282

In [11]:
pred = (1.0 + (y_pred == y_test))/2.0
hit_rate = np.mean(pred)
print('Logistic Regression - Hit Rate of 10 yr Treasury Yield Bond: {:.4f}'.format(hit_rate))

Logistic Regression - Hit Rate of 10 yr Treasury Yield Bond: 0.7504
